In [ ]:
!pip install transformers peft datasets torch evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType

In [ ]:
train_data = pd.read_csv('tamil_offensive_speech_train.csv', on_bad_lines='skip', encoding='utf-8')
print("Dataset loaded successfully:")
print(train_data.head())
train_dataset = Dataset.from_pandas(train_data)

Dataset loaded successfully:
   label                                            comment
0      0                  omg that bgm make me goosebumb...
1      0         neraya neraya neraya neraya neraya neraya.
2      0  thalaivar mersal look .semma massss thalaiva ....
3      0  paaaa... repeat mode.... adra adra adraaaaa......
4      0  epaa ena panaporam... sweet sapade poram... aw...


In [ ]:
import numpy as np
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from datasets import Dataset

# Extract features and labels
X_train = [example["comment"] for example in train_dataset]
y_train = [example["label"] for example in train_dataset]

# Convert lists to NumPy arrays
X_train = np.array(X_train).reshape(-1, 1)  # Reshape to (n_samples, 1)
y_train = np.array(y_train)

# Oversample minority class
ros = RandomOverSampler(sampling_strategy="auto", random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

# Convert back to Hugging Face Dataset
train_dataset = Dataset.from_dict({
    "comment": X_resampled.flatten().tolist(),  # Convert back to list of strings
    "label": y_resampled.tolist()
})

# Check class distribution after oversampling
print(Counter(y_resampled))  # Should show balanced classes



Counter({0: 21226, 1: 21226})


In [ ]:
val_data = pd.read_csv('tamil_offensive_speech_val.csv', on_bad_lines='skip', encoding='utf-8')
print("Dataset loaded successfully:")
print(val_data.head())
val_dataset = Dataset.from_pandas(val_data)

Dataset loaded successfully:
   label                                            comment
0      0  annan thambingalam poondhu vilayada poranga🤩🤩....
1      1                  ha ha ha ha appo naiya thebingala
2      0  manitha samuthaayam amaipil irunthu intha pada...
3      1  otha virundhalikku porandhavan tha jaadhi ah t...
4      1  ama padichavan vanda mattum vote potruvanungee...


In [ ]:
print(train_dataset[3])


{'comment': 'paaaa... repeat mode.... adra adra adraaaaa..... vera level... vare va....', 'label': 0}


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")

def tokenize_function(examples):
    comments = examples["comment"]

    # Ensure every comment is a string (convert None to an empty string)
    if isinstance(comments, list):
        comments = [c if isinstance(c, str) else "" for c in comments]
    else:
        comments = comments if isinstance(comments, str) else ""

    return tokenizer(comments, padding='max_length', truncation=True, max_length=128)



tamil_0_tokenized = train_dataset.map(tokenize_function, batched=False)

Map:   0%|          | 0/42452 [00:00<?, ? examples/s]

In [ ]:
tamil_1_tokenized = val_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/6969 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "google/muril-base-cased",
    num_labels=2
)


lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["query", "key", "value"],
    bias="none"
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import torch.nn as nn

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc}

In [ ]:
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
        """
        alpha: Balancing factor for class imbalance (list or tensor).
        gamma: Focusing parameter (higher = more focus on hard examples).
        reduction: 'mean' or 'sum' (same as CrossEntropyLoss).
        """
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, logits, targets):
        ce_loss = F.cross_entropy(logits, targets, reduction='none')  # Standard CE loss
        pt = torch.exp(-ce_loss)  # Probability of the true class
        focal_loss = (1 - pt) ** self.gamma * ce_loss  # Apply Focal Loss scaling

        # Apply alpha weighting if provided
        if self.alpha is not None:
            alpha_t = self.alpha.gather(0, targets.view(-1))  # Get alpha for each label
            focal_loss *= alpha_t

        # Reduction method
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        return focal_loss

In [ ]:
labels = np.array([example["label"] for example in train_dataset])
class_counts = np.bincount(labels)

class_weights = torch.tensor(1.0 / class_counts, dtype=torch.float).to("cuda")
class_weights = class_weights / class_weights.sum()
loss_fn = FocalLoss(alpha=class_weights, gamma=2.0)

In [ ]:
from transformers import Trainer, TrainingArguments


peft_model = get_peft_model(model, lora_config)
peft_model.print_trainable_parameters()

train_dataset = tamil_0_tokenized
val_dataset = tamil_1_tokenized


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=1)

    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="macro")  # Macro F1 handles class imbalance

    return {"accuracy": acc, "f1_score": f1}


class CustomTrainer(Trainer):
  def __init__(self, gamma=2.0, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.gamma = gamma

  def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
      labels = inputs.pop("labels")
      outputs = model(**inputs)
      logits = outputs.logits
      loss = loss_fn(logits, labels)
      return (loss, outputs) if return_outputs else loss

# Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=1e-4,  # Increase learning rate
    num_train_epochs=10,  # Increase epochs
    per_device_train_batch_size=64,  # Try larger batch
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Use the Custom Trainer
trainer = CustomTrainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    gamma=3.0
)

# Train the model
trainer.train()

# Save the model
trainer.save_model("./tamil-comment-classifier")

# Function for inference
def classify_comment(text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    model.eval()
    with torch.no_grad():
        outputs = peft_model(**inputs)

        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1).item()
    return "Offensive" if predicted_class == 1 else "Not Offensive"

# Test with a sample comment
sample_comment = "இது ஒரு நல்ல பதிவு"
prediction = classify_comment(sample_comment)
print(f"Comment: {sample_comment}")
print(f"Prediction: {prediction}")



trainable params: 886,274 || all params: 238,444,036 || trainable%: 0.3717


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-39-398e2bba2829>:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Accuracy,F1 Score
1,0.062500,0.059650,0.774717,0.737733
2,0.052600,0.062587,0.778447,0.744477
3,0.047500,0.053305,0.814034,0.769796
